In [1]:
import pandas as pd


import numpy as np
import pandas as pd

import seaborn as sns
from importlib import reload
import matplotlib.pyplot as plt
import preprocess_data 

reload(preprocess_data)

%matplotlib inline

In [2]:
measurements_blood_pressure = pd.read_csv('/data/public/MLA/VCHAMPS-Test/measurements_blood_pressure_test.csv').iloc[:,1:]

In [3]:
path = '/home/hassan/lily/MLA/FDA/inpatient_simple_test.csv'
inpatient = pd.read_csv(path).iloc[:,1:]
inpatient.shape

(22255, 41)

In [4]:
inpatient_date = inpatient[['Internalpatientid','Discharge date']]
inpatient_date

,Internalpatientid,Discharge date
0,7,2013-04-18 14:41:31.0
1,9,2000-04-10 02:38:04.0
2,12,2004-04-07 03:40:16.0
3,17,2004-05-05 15:18:05.0
4,22,2014-05-07 15:43:10.0
...,...,...
22250,168995,2010-06-09 10:03:21.0
22251,169011,2004-07-15 15:11:27.0
22252,169037,2015-02-12 00:48:40.0
22253,169059,2003-03-11 12:21:08.0


In [5]:
# Filter procedure date that's before discharge date
measurements_blood_pressure = measurements_blood_pressure.merge(inpatient_date, how = 'left', on = 'Internalpatientid')
measurements_blood_pressure = measurements_blood_pressure.loc[measurements_blood_pressure['Measurement date'] <= measurements_blood_pressure['Discharge date']]
measurements_blood_pressure 

,Internalpatientid,Age at measurement bp,Measurement date,Diastolic bp,Systolic bp,State,Discharge date
0,100,55.097387,2016-11-02 02:16:29.0,73.0,120.0,New York,2016-12-23 19:42:05.0
1,100,55.101809,2016-11-03 17:03:54.0,57.0,86.0,New York,2016-12-23 19:42:05.0
2,100,55.103167,2016-11-04 04:58:47.0,58.0,92.0,New York,2016-12-23 19:42:05.0
3,100,55.109996,2016-11-06 16:53:06.0,67.0,115.0,New York,2016-12-23 19:42:05.0
4,100,55.120935,2016-11-10 16:50:38.0,54.0,94.0,New York,2016-12-23 19:42:05.0
...,...,...,...,...,...,...,...
5918652,99990,80.593636,2003-08-06 23:22:42.0,73.0,146.0,Texas,2003-08-22 05:47:44.0
5918653,99990,80.607355,2003-08-11 23:43:10.0,65.0,107.0,Texas,2003-08-22 05:47:44.0
5918654,99990,80.612963,2003-08-14 00:54:44.0,52.0,120.0,Texas,2003-08-22 05:47:44.0
5918655,99990,80.615627,2003-08-15 00:16:52.0,48.0,114.0,Texas,2003-08-22 05:47:44.0


In [6]:
previous_df = inpatient_date

In [7]:
bins = [0, 100, 130, 200]

for i in range(1,2):
    data = measurements_blood_pressure['Internalpatientid'].loc[(measurements_blood_pressure['Systolic bp'] > i) & (measurements_blood_pressure['Age at measurement bp'] <= (i+1)*20)]
    new_df = pd.crosstab(data, pd.cut(measurements_blood_pressure['Systolic bp'], bins)).reset_index().fillna(0)
    new_df = new_df.rename(columns={pd.Interval(0, 60, closed='right'): 'Age ' + str(i*20) + '-' + str((i+1)*20)+ ' hypotension',
                                    pd.Interval(60, 80, closed='right'): 'Age ' + str(i*20) + '-' + str((i+1)*20) + ' healthy',
                                    pd.Interval(80, 200, closed='right'): 'Age ' + str(i*20) + '-' + str((i+1)*20) + ' hypetension'})
    previous_df = previous_df.merge(new_df, how = 'left', on = 'Internalpatientid')


In [8]:
# hypotension
bins = [20,40,60,80,100, 120]
previous_df = inpatient_date
data = measurements_blood_pressure['Internalpatientid'].loc[(measurements_blood_pressure['Systolic bp'] < 100) | (measurements_blood_pressure['Diastolic bp'] < 60)]
new_df = pd.crosstab(data, pd.cut(measurements_blood_pressure['Age at measurement bp'], bins)).reset_index().fillna(0)
new_df = new_df.rename(columns={pd.Interval(20, 40, closed='right'): 'Age 20-40 hypotension',
                                pd.Interval(40, 60, closed='right'): 'Age 40-60 hypotension',
                                pd.Interval(60, 80, closed='right'): 'Age 60-80 hypotension',
                                pd.Interval(80, 100, closed='right'): 'Age 80-100 hypotension',
                                pd.Interval(100, 120, closed='right'): 'Age 100-120 hypotension'})
previous_df = previous_df.merge(new_df, how = 'left', on = 'Internalpatientid')

In [9]:
# healthy
data = measurements_blood_pressure['Internalpatientid'].loc[(measurements_blood_pressure['Systolic bp'] >= 100) & (measurements_blood_pressure['Systolic bp'] < 130) 
                                                            & (measurements_blood_pressure['Diastolic bp'] >= 60) & (measurements_blood_pressure['Diastolic bp'] < 80)]
new_df = pd.crosstab(data, pd.cut(measurements_blood_pressure['Age at measurement bp'], bins)).reset_index().fillna(0)
new_df = new_df.rename(columns={pd.Interval(20, 40, closed='right'): 'Age 20-40 healthy',
                                pd.Interval(40, 60, closed='right'): 'Age 40-60 healthy',
                                pd.Interval(60, 80, closed='right'): 'Age 60-80 healthy',
                                pd.Interval(80, 100, closed='right'): 'Age 80-100 healthy',
                                pd.Interval(100, 120, closed='right'): 'Age 100-120 healthy'})
previous_df = previous_df.merge(new_df, how = 'left', on = 'Internalpatientid')

In [10]:
# hypertension
data = measurements_blood_pressure['Internalpatientid'].loc[(measurements_blood_pressure['Systolic bp'] > 130) | (measurements_blood_pressure['Diastolic bp'] > 80) ]
new_df = pd.crosstab(data, pd.cut(measurements_blood_pressure['Age at measurement bp'], bins)).reset_index().fillna(0)
new_df = new_df.rename(columns={pd.Interval(20, 40, closed='right'): 'Age 20-40 hypertension',
                                pd.Interval(40, 60, closed='right'): 'Age 40-60 hypertension',
                                pd.Interval(60, 80, closed='right'): 'Age 60-80 hypertension',
                                pd.Interval(80, 100, closed='right'): 'Age 80-100 hypertension',
                                pd.Interval(100, 120, closed='right'): 'Age 100-120 hypertension'})
previous_df = previous_df.merge(new_df, how = 'left', on = 'Internalpatientid')

In [11]:
previous_df = previous_df.drop(columns = ['Discharge date'])
previous_df.head()

,Internalpatientid,Age 20-40 hypotension,Age 40-60 hypotension,Age 60-80 hypotension,Age 80-100 hypotension,Age 100-120 hypotension,Age 20-40 healthy,Age 40-60 healthy,Age 60-80 healthy,Age 80-100 healthy,Age 100-120 healthy,Age 20-40 hypertension,Age 40-60 hypertension,Age 60-80 hypertension,Age 80-100 hypertension,Age 100-120 hypertension
0,7,0.0,4.0,4.0,0.0,0.0,0.0,63.0,126.0,0.0,0.0,0.0,48.0,154.0,0.0,0.0
1,9,0.0,8.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN
2,12,0.0,0.0,23.0,0.0,0.0,0.0,0.0,36.0,0.0,0.0,0.0,0.0,34.0,0.0,0.0
3,17,0.0,0.0,2.0,19.0,0.0,0.0,0.0,4.0,10.0,0.0,0.0,0.0,1.0,56.0,0.0
4,22,NaN,NaN,NaN,NaN,NaN,0.0,8.0,1.0,0.0,0.0,0.0,46.0,15.0,0.0,0.0


In [12]:
previous_df.to_csv('/home/vivi/FDA_datasets/inpatient_measurements_bp_test.csv')